In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('Data.csv')

In [3]:
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,03-01-2000,33.917892,35.945827,33.917892,35.945827,26.684868,31204789.0
1,04-01-2000,36.902668,38.823490,35.888702,38.823490,28.821136,66436055.0
2,05-01-2000,36.652744,41.115627,36.652744,40.344440,29.950230,187894923.0
3,06-01-2000,41.272720,42.943623,41.272720,42.036766,31.206558,109810562.0
4,07-01-2000,42.129593,45.399994,41.843967,44.921570,33.348133,139140630.0


In [5]:
df.shape

(6060, 7)

In [7]:
df.drop(columns=['Adj Close','Volume'],inplace= True)

In [8]:
df.head()

,Date,Open,High,Low,Close
0,03-01-2000,33.917892,35.945827,33.917892,35.945827
1,04-01-2000,36.902668,38.823490,35.888702,38.823490
2,05-01-2000,36.652744,41.115627,36.652744,40.344440
3,06-01-2000,41.272720,42.943623,41.272720,42.036766
4,07-01-2000,42.129593,45.399994,41.843967,44.921570


In [24]:
import numpy as np
import pandas as pd
from sklearn.svm import SVR
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error

# Assuming you have a CSV file containing historical stock data
data = pd.read_csv('Data.csv')
data.dropna(inplace=True)
# Extracting features: High, Low, Close
X = data[['High', 'Low', 'Close','Adj Close']].values

# Extracting labels: Open prices (shifted by one day to predict next day's opening price)
y = data['Open'].shift(-1).fillna(method='ffill').values

# Normalizing the features and labels
scaler_X = MinMaxScaler(feature_range=(0, 1))
scaler_y = MinMaxScaler(feature_range=(0, 1))
X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.reshape(-1, 1)).ravel()

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

# Define the SVR model
svr = SVR()

# Define the grid of parameters to search
param_grid = {
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'C': [0.1, 1, 10, 100],
    'gamma': ['scale', 'auto'],
    'epsilon': [0.1, 0.01, 0.001]
}

# Define GridSearchCV
grid_search = GridSearchCV(estimator=svr, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

# Perform GridSearchCV
grid_search.fit(X_train, y_train)

# Get the best parameters
best_params = grid_search.best_params_
print("Best Parameters:", best_params)

# Make predictions on the test set using the best model
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# Inverse transform the predictions and actual values to get the real stock prices
y_pred_inv = scaler_y.inverse_transform(y_pred.reshape(-1, 1)).ravel()
y_test_inv = scaler_y.inverse_transform(y_test.reshape(-1, 1)).ravel()

# Calculate Mean Squared Error
mse = mean_squared_error(y_test_inv, y_pred_inv)
print("Mean Squared Error:", mse)


Best Parameters: {'C': 100, 'epsilon': 0.001, 'gamma': 'scale', 'kernel': 'linear'}
Mean Squared Error: 71.05966529020134


In [22]:
from sklearn.metrics import r2_score
y_pred = model.predict(X_test)
r2_score(y_test,y_pred)

38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


-1.2583920171893581e-05

In [23]:
import pickle
with open('model.pkl', 'wb') as f:
    pickle.dump(best_model, f)

In [34]:
value = np.array([[2972,2918.7,2923.15,2931.5],
[2948,2886.05,2937.45,2928.65],
[2964,2935.6,2962.85,2940.25],
[2987,2913.05,2915.5,2959.7],
[2937.1,2899,2901.95,2918.65],
[2935.95,2883,2913.2,2900.35],
[2930,2900,2907.8,2919.95]])



In [26]:
df2 = pd.read_csv('PREDICTION - Sheet1.csv')

In [27]:
df2.head()

,DATE,OPEN,HIGH,LOW,CLOSE ( 3.20 ),PREV CLOSE,PREDICTION ( OPEN ),ACTUAL OPEN,DIFFRENCE BETWEEN ACTUAL & PREDICTION
0,01-04-2024,2984.95,2987.95,2965.0,2969.60,2971.70,2990.243702,2968.00,22.243702
1,02-04-2024,2968.00,2988.00,2950.0,2975.85,2969.55,2992.292147,2964.15,28.142147
2,03-04-2024,2964.15,2968.90,2937.8,2939.90,2973.90,2979.622980,2959.90,19.722980
3,04-04-2024,2959.90,2959.90,2900.0,2926.75,2943.20,2959.242521,2921.75,37.492521
4,05-04-2024,2921.75,2941.60,2912.0,2919.10,2925.85,2958.059032,2925.95,32.109032
